# Train a Keras Model in Pipe Mode
This notebook shows how to train and host a Keras Sequential model on SageMaker. The model used for this notebook is a simple deep CNN that was extracted from [the Keras examples](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py).

## The dataset
The [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) is one of the most popular machine learning datasets. It consists of 60,000 32x32 images belonging to 10 different classes (6,000 images per class). Here are the classes in the dataset, as well as 10 random images from each:

![cifar10](https://maet3608.github.io/nuts-ml/_images/cifar10.png)

In this tutorial, we will train a deep CNN to recognize these images.

We'll compare trainig with file mode, pipe mode datasets and distributed training with Horovod

## Set up the environment

In [14]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

In [16]:
bucket = '<BUCKET NAME>'
prefix = 'sagemaker/script-mode-pipe'

## Download the CIFAR-10 dataset
Downloading the test and training data takes around 5 minutes.

In [17]:
#!pip install wget
# import wget # for TF2

In [18]:
!mkdir data
!python generate_cifar10_tfrecords_v2.py --data-dir data/

mkdir: cannot create directory 'data': File exists
Download from https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz and extract.
data/
100% [..................................................] 170498071 / 170498071Generating data//train/train.tfrecords
Generating data//validation/validation.tfrecords
Generating data//eval/eval.tfrecords
Done!


## Run on SageMaker cloud

### Uploading the data to s3

In [8]:
dataset_location = sagemaker_session.upload_data(bucket=bucket, path='data', key_prefix=prefix)
display(dataset_location)

's3://demo-saeed/sagemaker/script-mode-pipe'

### Configuring metrics from the job logs
SageMaker can get training metrics directly from the logs and send them to CloudWatch metrics.

In [9]:
keras_metric_definition = [
    {'Name': 'train:loss', 'Regex': '.*loss: ([0-9\\.]+) - acc: [0-9\\.]+.*'},
    {'Name': 'train:accuracy', 'Regex': '.*loss: [0-9\\.]+ - acc: ([0-9\\.]+).*'},
    {'Name': 'validation:accuracy', 'Regex': '.*step - loss: [0-9\\.]+ - acc: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_acc: ([0-9\\.]+).*'},
    {'Name': 'validation:loss', 'Regex': '.*step - loss: [0-9\\.]+ - acc: [0-9\\.]+ - val_loss: ([0-9\\.]+) - val_acc: [0-9\\.]+.*'},
    {'Name': 'sec/steps', 'Regex': '.* - \d+s (\d+)[mu]s/step - loss: [0-9\\.]+ - acc: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_acc: [0-9\\.]+'}
]

### Train image classification based on the cifar10 dataset

In [10]:
hyperparameters = {'epochs': 10, 'batch-size' : 256}

## Run on SageMaker with Pipe Mode input
SageMaker Pipe Mode is a mechanism for providing S3 data to a training job via Linux fifos. Training programs can read from the fifo and get high-throughput data transfer from S3, without managing the S3 access in the program itself.
Pipe Mode is covered in more detail in the SageMaker [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html#your-algorithms-training-algo-running-container-inputdataconfig)

in out script, we enabled Pipe Mode using the following code:
```python
from sagemaker_tensorflow import PipeModeDataset
dataset = PipeModeDataset(channel=channel_name, record_format='TFRecord')
```

In [11]:
from sagemaker.tensorflow import TensorFlow


source_dir = os.path.join(os.getcwd(), 'source_dir')
estimator_pipe = TensorFlow(base_job_name='pipe-cifar10-tf',
                       entry_point='cifar10_keras_main.py',
                       source_dir=source_dir,
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       hyperparameters=hyperparameters,
                       train_instance_count=1, train_instance_type='ml.p3.2xlarge',
                       metric_definitions=keras_metric_definition,
                       input_mode='Pipe')

In this example, we set ```wait=False``` if you want to see the output logs, change this to ```wait=True```

In [13]:
remote_inputs = {'train' : dataset_location+'/train', 'validation' : dataset_location+'/validation', 'eval' : dataset_location+'/eval'}
estimator_pipe.fit(remote_inputs, wait=True)

2020-06-23 20:31:16 Starting - Starting the training job...
2020-06-23 20:31:18 Starting - Launching requested ML instances......
2020-06-23 20:32:35 Starting - Preparing the instances for training......
2020-06-23 20:33:42 Downloading - Downloading input data...
2020-06-23 20:33:58 Training - Downloading the training image..2020-06-23 20:34:23,185 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-06-23 20:34:30,960 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "batch-size": 256,
        "model_dir": "s3://sagemaker-us-east-1-0793291